In [17]:
#imports
import numpy as np
import random
from functools import reduce

In [18]:
# parity check
def isEvenParity(l):
    s = 0
    for x in l:
        s += x
    if (s % 2 == 0):
        return True
    else:
        return False

In [19]:
# setup hamming 16 bits matrix
def createHamming16bits(m):
    # definition de variable utiles
    firstParity = [0, 1, 3, 4, 6, 8, 10]
    secondParity = [0, 2, 3, 5, 6, 9, 10]
    thirdParity = [1, 2, 3, 7, 8, 9, 10]
    fourthParity = [4, 5, 6, 7, 8, 9, 10]
    # création de la matrice
    matrixHamming = m.copy()
    # parité 1
    l = [matrixHamming[i] for i in range(len(matrixHamming)) if i in firstParity]
    p1 = 0 if isEvenParity(l) else 1
    # parité 2
    l = [matrixHamming[i] for i in range(len(matrixHamming)) if i in secondParity]
    p2 = 0 if isEvenParity(l) else 1
    # parité 3
    l = [matrixHamming[i] for i in range(len(matrixHamming)) if i in thirdParity]
    p3 = 0 if isEvenParity(l) else 1
    # parité 4
    l = [matrixHamming[i] for i in range(len(matrixHamming)) if i in fourthParity]
    p4 = 0 if isEvenParity(l) else 1
    matrixHamming.insert(0, p1)
    matrixHamming.insert(1, p2)
    matrixHamming.insert(3, p3)
    matrixHamming.insert(7, p4)
    # parité totale
    pt = 0 if isEvenParity(matrixHamming) else 1
    matrixHamming.insert(0, pt)
    return matrixHamming

In [20]:
#check Parity in Hamming matrix
def isValidHamming16bits(m):
    if (len(m) != 16):
        return False
    if (reduce(lambda x, y : x + y, [m[i] for i in range(len(m)) if i in [1, 5, 9, 13, 3, 7, 11, 15]]) % 2 != 0):
     return False
    if (reduce(lambda x, y : x + y, [m[i] for i in range(len(m)) if i in [2, 3, 6, 7, 10, 11, 14, 15]]) % 2 != 0):
     return False
    if (reduce(lambda x, y : x + y, [m[i] for i in range(len(m)) if i in [4, 5, 6, 7, 12, 13, 14, 15]]) % 2 != 0):
     return False
    if (reduce(lambda x, y : x + y, [m[i] for i in range(len(m)) if i in [8, 9, 10, 11, 12, 13, 14, 15]]) % 2 != 0):
     return False
    return True

In [21]:
# Read File
byteFile = []
filePath = "img.jpg"
with open(filePath, "rb") as f:
    byte = f.read(1)
    while byte != b"":
        byteFile.append(byte)
        byte = f.read(1)

In [22]:
# convert bytes to bits
binFile = []
for b in byteFile:
    i = int.from_bytes(b, byteorder='big', signed=False)
    for x in [int(x) for x in bin(i)[2:]]:
        binFile.append(x)

In [23]:
# converting to Hamming
arrayHamming = []
n = 0
matrixHamming = []
for binValue in binFile:
    if (len(matrixHamming) == 11):
        arrayHamming.append(createHamming16bits(matrixHamming))
        # reset matrix
        matrixHamming = []
    matrixHamming.append(binValue)

In [24]:
# missing bits setup
missingBits = 11 - len(matrixHamming)
for i in range(missingBits):
        matrixHamming.append(0)
arrayHamming.append(createHamming16bits(matrixHamming))

In [25]:
# ajout missing bits
binValue = [int(x) for x in bin(i)[2:]]
for i in range(11 - len(binValue)):
    binValue.insert(0, 0)
matrixHamming = binValue
arrayHamming.append(createHamming16bits(matrixHamming))

In [26]:
# shifting bits data sent to limit error on single hamming matrix
sentData = []
for i in range(16):
    for x in arrayHamming:
        sentData.append(x[i])

In [27]:
# introducing noise
onlyError = sentData.copy()
# index
onlyErrorIndex = random.randint(0, len(sentData) - 1)
print(onlyErrorIndex)
# introducing error
onlyError[onlyErrorIndex] = not onlyError[onlyErrorIndex]

138255


In [28]:
# trad to hamming
rcv = []
for i in range(len(sentData) // 16):
    rcv.append([])
for i in range(len(onlyError)):
    rcv[i%(len(rcv))].append(onlyError[i])

In [29]:
# detection erreur
n = 0
for matrix in rcv:
    ValidParity = reduce(lambda x, y : y + x, matrix) % 2 == 0
    if (not ValidParity):
        xorRes = reduce(operator.xor, [i for i , bit in enumerate(matrix) if bit])
        print("error matrice {:d}".format(n))
        print(xorRes)
    else :
        if (not isValidHamming16bits(matrix)):
            print("Deux erreurs dans la matrice {:n}".format(n))
    n += 1

error matrice 56010
1


In [30]:
# introducing noise
multipleError = sentData.copy()
# index
errorTuple = (random.randint(0, len(sentData) - 10), random.randint(2, 9))
# introducing errors
for x in range(errorTuple[1]):
    multipleError[errorTuple[0] + x] = not multipleError[errorTuple[0] + x]
print(errorTuple)

(592496, 3)


In [31]:
# trad to hamming
rcv = []
for i in range(len(sentData) // 16):
    rcv.append([])
for i in range(len(multipleError)):
    rcv[i%(len(rcv))].append(multipleError[i])

In [32]:
# detection erreur
n = 0
for matrix in rcv:
    ValidParity = reduce(lambda x, y : y + x, matrix) % 2 == 0
    if (not ValidParity):
        xorRes = reduce(operator.xor, [i for i , bit in enumerate(matrix) if bit])
        print("error matrice {:d}".format(n))
        print(xorRes)
    else :
        if (not isValidHamming16bits(matrix)):
            print("Deux erreurs dans la matrice {:n}".format(n))
    n += 1

error matrice 16781
7
error matrice 16782
7
error matrice 16783
7
